In [10]:
!pip install langchain-groq chromadb langchain langchain_community
import langchain

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='gsk_9j4Ef3Q0e8aZT8dbswfg46udcbgswrtauabqwt675rws',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first president of Bangladesh ...")
print(response.content)

Sheikh Mujibur Rahman was the first president of Bangladesh.


In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.bat.com/job/bangladesh/expression-of-interest-territory-officer/27325/16749958016")
page_data = loader.load().pop().page_content
print(page_data)









Expression of Interest: Territory Officer








































































































Skip to main content














Just for you
Stay in touch
Sign in






 






English
Deutsch
Español
Français
日本語
bahasa Indonesia
Polski
Română
한국어
Português do Brasil
Türkçe
Українська
Русский
中文
中國人


















Just for you
Stay in touch
Sign in
Saved Jobs


Our business
Culture

Opportunities

Professionals
Graduates
Internships


Selection approach
Our Stories








What?



Where?



Search












Expression of Interest: Territory Officer









Save 
Apply

BAT is evolving at pace - truly like no other organisation. To achieve the ambition, we have set for ourselves, we are looking for colleagues who are ready to live our values every day. Come be a part of this journey!BATBANGLADESHIS LOOKING FOR A TERRITORY OFFICER  SENIORITY LEVEL: Entry LevelFUNCTION: Marketing & CommercialSALARY RANGE: Competitive sal

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [25]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Territory Officer',
  'experience': 'Entry Level',
  'skills': ['A sound educational background with minimum Bachelors degree (preferably business graduates)',
   'A customer focused approach & solutions driven mentality',
   'Strong selling, negotiation, and objective handling skills',
   'The aim to be empowered, work independently and focus on results',
   'A proven track record to build close working relationships with a diverse range of partners',
   'Excellent planning and prioritisation skills'],
  'description': 'BATBANGLADESHIS LOOKING FOR A TERRITORY OFFICER. Analysing, reporting, and interpreting store specific data in order to drive growth. Acting as the point of reference for external partners by engaging in commercial conversations and negotiations. Building, encouraging, and maintaining strong business partnerships with retailers. Responsibly educating retailers on product and category knowledge as well as on trading terms. Ensuring execution of strategies and

In [26]:
type(json_res)

list

In [27]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

df

Saving my_portfolio.csv to my_portfolio.csv


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


No charts were generated by quickchart


In [29]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])



In [36]:
links = collection.query(query_texts=["Experience in DevOps"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [41]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Rafid, a business development executive at Dohatec. Dohatec is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Dohatec
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Dohatec's portfolio: {link_list}
        Remember you are Rafid, BDE at Dohatec.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Business Potential with Dohatec's AI & Software Solutions

Dear Hiring Manager,

I came across the job postings for Territory Officer and Global Graduate - Marketing & Commercial at BAT Bangladesh, and I was impressed by the company's commitment to driving growth and building strong business partnerships. As a Business Development Executive at Dohatec, I believe our AI & Software Consulting services can help empower your team to achieve their goals.

At Dohatec, we specialize in facilitating the seamless integration of business processes through automated tools. Our expertise in process optimization, cost reduction, and scalability can help your team streamline operations, enhance customer engagement, and ultimately drive revenue growth. Our solutions are tailored to meet the unique needs of each client, ensuring maximum ROI and efficiency.

Our portfolio showcases our capabilities in DevOps and Full-Stack JavaScript development, which can be leveraged to enhance you